In [1]:
import torch
import numpy as np
import torch.nn as nn

In [50]:
x = np.linspace(0,31,32)
x = x.reshape(2, 4, 2, 2)
x = torch.from_numpy(x)

In [ ]:
z = x

z = z.permute(0, 2, 3, 1)
z = z.reshape(-1, 4)

tensor([ 0.,  4.,  8., 12.], dtype=torch.float64)

In [53]:
codebook = nn.Embedding(10, 4)
nn.init.uniform_(codebook.weight, -1.0 / 10, 1.0 / 10)

Parameter containing:
tensor([[-0.0219,  0.0001,  0.0317,  0.0032],
        [ 0.0455, -0.0334, -0.0576,  0.0350],
        [-0.0258, -0.0957, -0.0685, -0.0249],
        [-0.0777, -0.0137,  0.0223, -0.0190],
        [ 0.0186,  0.0719,  0.0275,  0.0206],
        [-0.0668,  0.0338,  0.0956, -0.0427],
        [ 0.0999, -0.0972, -0.0757,  0.0092],
        [-0.0633, -0.0296, -0.0101, -0.0694],
        [ 0.0524,  0.0302,  0.0141, -0.0548],
        [-0.0467,  0.0553, -0.0149,  0.0004]], requires_grad=True)

In [ ]:
distances = torch.sum(z**2, dim=1, keepdim=True) + \
            torch.sum(codebook.weight**2, dim=1) - \
            2 * torch.matmul(z, codebook.weight.t())

tensor([[0.0000, 0.1211, 0.1415, 0.0623, 0.0843, 0.0966, 0.1894, 0.0980, 0.1005,
         0.0764],
        [0.1211, 0.0000, 0.1126, 0.1577, 0.1388, 0.2160, 0.0896, 0.1581, 0.1315,
         0.1392],
        [0.1415, 0.1126, 0.0000, 0.1331, 0.2033, 0.2138, 0.1304, 0.1056, 0.1722,
         0.1635],
        [0.0623, 0.1577, 0.1331, 0.0000, 0.1349, 0.0911, 0.2212, 0.0637, 0.1421,
         0.0865],
        [0.0843, 0.1388, 0.2033, 0.1349, 0.0000, 0.1319, 0.2144, 0.1628, 0.0935,
         0.0822],
        [0.0966, 0.2160, 0.2138, 0.0911, 0.1319, 0.0000, 0.2775, 0.1262, 0.1450,
         0.1223],
        [0.1894, 0.0896, 0.1304, 0.2212, 0.2144, 0.2775, 0.0000, 0.2041, 0.1750,
         0.2202],
        [0.0980, 0.1581, 0.1056, 0.0637, 0.1628, 0.1262, 0.2041, 0.0000, 0.1332,
         0.1113],
        [0.1005, 0.1315, 0.1722, 0.1421, 0.0935, 0.1450, 0.1750, 0.1332, 0.0000,
         0.1198],
        [0.0764, 0.1392, 0.1635, 0.0865, 0.0822, 0.1223, 0.2202, 0.1113, 0.1198,
         0.0000]], grad_fn=<

In [7]:
self_filters = 128
resolution = 128
layers = []
channel_multipliers = [1, 1, 2, 2, 4]
# x = conv_fn(filters, kernel_size=(3, 3), use_bias=False)(x)
layers.append(f"initial conv c = {self_filters} r = {resolution}")
num_blocks = len(channel_multipliers)
for i in range(num_blocks):
    filters = self_filters * channel_multipliers[i]
    for _ in range(1):
    # x = ResBlock(filters, **block_args)(x)
        layers.append(f"resblocks c = {filters} r = {resolution}")
    if i < num_blocks - 1:
    # x = conv_fn(filters, kernel_size=(4, 4), strides=(2, 2))(x)
        resolution = resolution / 2
        layers.append(f"downsample c = {filters} r = {resolution}\n")
for _ in range(1):
#   x = ResBlock(filters, **block_args)(x)
    layers.append(f"resblocks c = {filters} r = {resolution}")
# x = norm_fn()(x)
# x = activation_fn(x)
# x = conv_fn(embedding_dim, kernel_size=(1, 1))(x)
layers.append(f"final convolution c = {512} r = {resolution}")
for l in layers:
    print(l)

initial conv c = 128 r = 128
resblocks c = 128 r = 128
downsample c = 128 r = 64.0

resblocks c = 128 r = 64.0
downsample c = 128 r = 32.0

resblocks c = 256 r = 32.0
downsample c = 256 r = 16.0

resblocks c = 256 r = 16.0
downsample c = 256 r = 8.0

resblocks c = 512 r = 8.0
resblocks c = 512 r = 8.0
final convolution c = 512 r = 8.0


In [13]:
# This will all go in a config file later
num_res_blocks = 2
data_channels = 1
filters = 128
channel_multipliers = [1, 1, 2, 2, 4]
latent_dim = 256
resolution = 128
# ------

# first_conv = nn.Conv2d(data_channels, filters, kernel_size = 3, stride = 1, padding = 1)
first_conv = f"conv c= {filters} r = {resolution}"
layers = [first_conv]


# So I could either have the ResBlock multiply the channels or I could have the downsample layer do it
# I'm going with the latter because it should be less computationally expensive
# But the former could potentially be more expressive as the second ResBlock would get to work at a deeper channel depth with the full resolution

# Another thing to note is that I'm not really sure if the downsample layer goes before or after each residual block. I have it after.
num_blocks = len(channel_multipliers)
for i in range(num_blocks):
    in_channels = filters * channel_multipliers[i]

    for _ in range(1):
        # layers.append(ResBlock(in_channels, in_channels))
        layers.append(f"resblock c = {in_channels} r = {resolution}")

    if i < num_blocks - 1:
        # Halves the resolution and multiplies the channels according to the channel_multipliers
        resolution = resolution / 2
        out_channels = filters * channel_multipliers[i]
        # downsample_layer = nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = 2, padding = 2)
        layers.append(f"downsample c = {out_channels} r = {resolution}\n")

for _ in range(1):
    final_channels = filters * channel_multipliers[-1]
    layers.append(f"resblock c = {final_channels} r = {resolution}")

# layers.append(nn.BatchNorm2D(final_channels))
# layers.append(nn.ReLu())
# layers.append(nn.Conv2d(final_channels, latent_dim, kernel_size = 1, stride = 1, padding = 1)) # Squish hidden dim back down to latent dim

# self.model = nn.Sequential(*layers)
for l in layers:
    print(l)

conv c= 128 r = 128
resblock c = 128 r = 128
downsample c = 128 r = 64.0

resblock c = 128 r = 64.0
downsample c = 128 r = 32.0

resblock c = 256 r = 32.0
downsample c = 256 r = 16.0

resblock c = 256 r = 16.0
downsample c = 256 r = 8.0

resblock c = 512 r = 8.0
resblock c = 512 r = 8.0
